# **Summarization Using a Causal Model**

---




In [ ]:
!pip install datasets

In [3]:
from datasets import load_dataset

In [ ]:
hf_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(hf_dataset_name)

# Testing **Qwen2.5-3B-Instruct**

Since it is a **decoder-only (causal)** LLM, it’s not a native seq2seq model, but it can be still used it for seq2seq tasks by **formatting** the input as a **“prompt → desired output”** sequence.

One can simulate **seq2seq-style behavior** using prompting (**instruction format**), but internally it is still decoder-only.

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [6]:
model_id = "Qwen/Qwen2.5-3B-Instruct"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype="auto",
    device_map="auto" # ensures it uses GPU if available
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following dialogue as short as possible:

{dialogue}
"""

In [10]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

In [19]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [20]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [25]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=50
)

In [26]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [27]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [28]:
print(f'Dialogue: \n{dialogue}\n')

print(f'Generated Summary: {response}\n')

print(f'Human Summary: {summary}')

Dialogue: 
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Generated Summary: Person1 suggests Person2 consider upgrading their system by adding a painting program for creating flyers and banners. Person1 also recommends upgrading hardware, including a faster processor, more pow